# BIG DATA AND CLOUD COMPUTING - PROJECT


## Authentication

In [1]:
PROJECT_ID = 'proj8824'


In [2]:
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {PROJECT_ID}


Updated property [core/project].


## Big Query Dataset Creation

In [5]:
from google.cloud import bigquery as bq

client = bq.Client(project=PROJECT_ID)


### Tables Creation

In [6]:

DATASET_ID = "openimages"

# Dataset creation
dataset_ref = client.dataset(DATASET_ID)
dataset = bq.Dataset(dataset_ref)
dataset.location = "US"
client.create_dataset(dataset, exists_ok=True)

# Schema for Classes table
classes_schema = [
    bq.SchemaField("Label", "STRING", mode="REQUIRED"),
    bq.SchemaField("Description", "STRING", mode="REQUIRED"),
]

# Define schema for image_labels table
image_labels_schema = [
    bq.SchemaField("ImageId", "STRING", mode="REQUIRED"),
    bq.SchemaField("Label", "STRING", mode="REQUIRED"),
]

# Schema for relations table
relations_schema = [
    bq.SchemaField("ImageId", "STRING", mode="REQUIRED"),
    bq.SchemaField("Label1", "STRING", mode="REQUIRED"),
    bq.SchemaField("Relation", "STRING", mode="REQUIRED"),
    bq.SchemaField("Label2", "STRING", mode="REQUIRED"),
]

# Table creation
tables = {
    'classes': classes_schema,
    'image_labels': image_labels_schema,
    'relations': relations_schema
}

for table_id, schema in tables.items():
    table_ref = dataset_ref.table(table_id)
    table = bq.Table(table_ref, schema=schema)
    client.create_table(table, exists_ok=True)

print("BigQuery dataset and tables created successfully.")


BigQuery dataset and tables created successfully.


### Loading Data Into BigQuery

In [10]:
import pandas as pd

image_labels_df = pd.read_csv('image-labels.csv')
classes_df = pd.read_csv('classes.csv')
relations_df = pd.read_csv('relations.csv')

In [11]:
# Loading data from DataFrame to BigQuery table
def load_data_to_bigquery(df, table_id):
    job_config = bq.LoadJobConfig(autodetect=True)  # Let BigQuery determine the schema automatically
    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
    job.result()  # Wait for the job to complete

# Loading the 'classes' data
classes_table_id = f"{PROJECT_ID}.openimages.classes"
load_data_to_bigquery(classes_df, classes_table_id)

# Loading the 'image_labels' data
image_labels_table_id = f"{PROJECT_ID}.openimages.image_labels"
load_data_to_bigquery(image_labels_df, image_labels_table_id)

# Loading the 'relations' data
relations_table_id = f"{PROJECT_ID}.openimages.relations"
load_data_to_bigquery(relations_df, relations_table_id)

print("Data has been uploaded to BigQuery.")


Data has been uploaded to BigQuery.


### Validating the Data Loading

In [12]:
# Comparing the row counts for each table
for table_name, dataframe in [('classes', classes_df), ('image_labels', image_labels_df), ('relations', relations_df)]:
    table_id = f"{PROJECT_ID}.{DATASET_ID}.{table_name}"
    table = client.get_table(table_id)  # Make an API request.
    print(f"The table {table_id} contains {table.num_rows} rows.")

    expected_row_count = len(dataframe.index)
    print(f"The DataFrame for {table_name} contains {expected_row_count} rows.")

    if expected_row_count == table.num_rows:
        print(f"Row count matches for {table_id}!")
    else:
        print(f"Row count mismatch for {table_id}!")


The table proj8824.openimages.classes contains 601 rows.
The DataFrame for classes contains 601 rows.
Row count matches for proj8824.openimages.classes!
The table proj8824.openimages.image_labels contains 550809 rows.
The DataFrame for image_labels contains 550809 rows.
Row count matches for proj8824.openimages.image_labels!
The table proj8824.openimages.relations contains 2768 rows.
The DataFrame for relations contains 2768 rows.
Row count matches for proj8824.openimages.relations!


In [13]:
# Checking the first few rows
for table_name in ['classes', 'image_labels', 'relations']:
    query = f"SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.{table_name}` LIMIT 5"
    query_job = client.query(query)

    print(f"Showing sample data for {table_name}:")
    for row in query_job.result():
        print(dict(row))
    print("\n")


Showing sample data for classes:
{'Label': '/m/011k07', 'Description': 'Tortoise'}
{'Label': '/m/011q46kg', 'Description': 'Container'}
{'Label': '/m/012074', 'Description': 'Magpie'}
{'Label': '/m/0120dh', 'Description': 'Sea turtle'}
{'Label': '/m/01226z', 'Description': 'Football'}


Showing sample data for image_labels:
{'ImageId': '01054f0eadfb2d5a', 'Label': '/m/0_k2'}
{'ImageId': '0d3786516a4d335e', 'Label': '/m/0_k2'}
{'ImageId': '14da3cbe7b9bd730', 'Label': '/m/0_k2'}
{'ImageId': '1564ad0dd401481c', 'Label': '/m/0_k2'}
{'ImageId': 'c07604d8bfb3b93c', 'Label': '/m/0_k2'}


Showing sample data for relations:
{'ImageId': '214a5095725a829d', 'Label1': '/m/01yrx', 'Relation': 'inside_of', 'Label2': '/m/0k4j'}
{'ImageId': 'e946e9130ec264a6', 'Label1': '/m/01yrx', 'Relation': 'inside_of', 'Label2': '/m/0k4j'}
{'ImageId': 'f89aed52e139832b', 'Label1': '/m/04yx4', 'Relation': 'inside_of', 'Label2': '/m/0k4j'}
{'ImageId': '2d9c4a7383cbbf33', 'Label1': '/m/04yx4', 'Relation': 'inside_of'

In [14]:
# Checking the schema
for table_name in ['classes', 'image_labels', 'relations']:
    table_id = f"{PROJECT_ID}.{DATASET_ID}.{table_name}"
    table = client.get_table(table_id)  # Make an API request.
    print(f"Schema for {table_id}:")
    for schema_field in table.schema:
        print(f"{schema_field.name} - {schema_field.field_type}")


Schema for proj8824.openimages.classes:
Label - STRING
Description - STRING
Schema for proj8824.openimages.image_labels:
ImageId - STRING
Label - STRING
Schema for proj8824.openimages.relations:
ImageId - STRING
Label1 - STRING
Relation - STRING
Label2 - STRING


## Creating CSV for Vertex AI

In [8]:
from google.cloud import bigquery
from google.cloud import storage
import pandas as pd


PROJECT_ID = 'proj8824'
BQ_CLIENT = bigquery.Client(project=PROJECT_ID)
STORAGE_CLIENT = storage.Client(project=PROJECT_ID)


SOURCE_BUCKET_NAME = 'bdcc_open_images_dataset'
TARGET_BUCKET_NAME = 'proj8824.appspot.com'


DATASET_ID = 'openimages'
IMAGES_TABLE_NAME = 'image_labels'
LABELS_TABLE_NAME = 'classes'

# Specifying labels to include
INCLUDE_LABEL_DESCRIPTIONS = [
    'Chair', 'Desk', 'Bed', 'Bookcase', 'Door handle', 'Nightstand', 'Couch', 'Houseplant', 'Picture frame', 'Lamp'
]

# Query to fetch images, considering additional non-specified labels are acceptable
query = f"""
SELECT IL.ImageId, CL.Description
FROM `{PROJECT_ID}.{DATASET_ID}.{IMAGES_TABLE_NAME}` AS IL
JOIN `{PROJECT_ID}.{DATASET_ID}.{LABELS_TABLE_NAME}` AS CL ON IL.Label = CL.Label
WHERE CL.Description IN UNNEST(@include_labels)
GROUP BY IL.ImageId, CL.Description
"""

job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("include_labels", "STRING", INCLUDE_LABEL_DESCRIPTIONS)
    ]
)

query_job = BQ_CLIENT.query(query, job_config=job_config)
results = query_job.result()

# Organizing images by their label, ensuring uniqueness across categories
used_images = set()
images_by_label = {label: [] for label in INCLUDE_LABEL_DESCRIPTIONS}

for row in results:
    image_id = row["ImageId"]
    label = row["Description"]
    if image_id not in used_images and len(images_by_label[label]) < 100:
        images_by_label[label].append(image_id)
        used_images.add(image_id)  # Mark this image as used

# Copying an image from the source to the target bucket and returning the target URI
def copy_image_to_target_bucket(source_image_id):
    source_blob_path = f"images/{source_image_id}.jpg"
    target_blob_path = f"ImageFiles/{source_image_id}.jpg"
    source_blob = STORAGE_CLIENT.bucket(SOURCE_BUCKET_NAME).blob(source_blob_path)
    target_blob = STORAGE_CLIENT.bucket(TARGET_BUCKET_NAME).copy_blob(source_blob, STORAGE_CLIENT.bucket(TARGET_BUCKET_NAME), target_blob_path)
    return f"gs://{TARGET_BUCKET_NAME}/{target_blob_path}"

# Prepareing the CSV data
csv_data = []

for label, images in images_by_label.items():
    for idx, image_id in enumerate(images):
        # Determining the dataset split (training, validation, test) based on the index
        if idx < 80:
            data_type = 'training'
        elif idx < 90:
            data_type = 'validation'
        else:
            data_type = 'test'

        # Copying the image to our bucket and getting the target URI
        target_image_uri = copy_image_to_target_bucket(image_id)

        # Appending the data to the CSV list
        csv_data.append({
            "set": data_type,
            "image_uri": target_image_uri,
            "label": label
        })

# Creating a dataframe
df = pd.DataFrame(csv_data)

# Saving the DataFrame to a CSV file
csv_file_path = "vertex_ai_dataset.csv"
df.to_csv(csv_file_path, index=False)

print(f"CSV file created at {csv_file_path}")


CSV file created at vertex_ai_dataset.csv
